In [2]:
import pandas as pd

In [4]:
df= pd.read_csv(r"X:\BIA\Practise\Datasets\reviews.csv", nrows= 2000)

In [5]:
df.head()

,course_code,course_title,num_ratings,useful,easy,liked,num_reviews,reviews,course_rating
0,CS 115,Introduction to Computer Science 1,2111,21%,10%,23%,86,go to office hours and practice,liked course
1,CS 115,Introduction to Computer Science 1,2111,21%,10%,23%,86,One of my least favourite courses. Although th...,disliked course
2,CS 115,Introduction to Computer Science 1,2111,21%,10%,23%,86,It starts with a very low pace but after midte...,disliked course
3,CS 115,Introduction to Computer Science 1,2111,21%,10%,23%,86,Took this in 2018 with no programming experien...,liked course
4,CS 115,Introduction to Computer Science 1,2111,21%,10%,23%,86,I loved everything about cs 115. Great instruc...,liked course


In [6]:
import string
from nltk.corpus import wordnet, stopwords
from nltk import pos_tag
from nltk.tokenize import WhitespaceTokenizer
from nltk.stem import WordNetLemmatizer

In [7]:
## return the wordnet object value corresponding to the POS tag
def get_wordnet_pos(pos_tag):
    if pos_tag.startswith('J'):
        return wordnet.ADJ
    elif pos_tag.startswith('V'):
        return wordnet.VERB
    elif pos_tag.startswith('N'):
        return wordnet.NOUN
    elif pos_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN
    

## Clean the text
def clean_text(text):
    # lower text
    text = text.lower()
    # tokenize text and remove puncutation
    text = [word.strip(string.punctuation) for word in text.split(" ")]
    # remove words that contain numbers
    text = [word for word in text if not any(c.isdigit() for c in word)]
    # remove stop words
    stop = stopwords.words('english')
    text = [x for x in text if x not in stop]
    # remove empty tokens
    text = [t for t in text if len(t) > 0]
    # pos tag text
    pos_tags = pos_tag(text)
    # lemmatize text
    text = [WordNetLemmatizer().lemmatize(t[0], get_wordnet_pos(t[1])) for t in pos_tags]
    # remove words with only one letter
    text = [t for t in text if len(t) > 1]
    # join all
    text = " ".join(text)
    return(text)

In [8]:
df['reviews']

0                         go to office hours and practice
1       One of my least favourite courses. Although th...
2       It starts with a very low pace but after midte...
3       Took this in 2018 with no programming experien...
4       I loved everything about cs 115. Great instruc...
                              ...                        
1995    Most useful course in CS undergrad. Assignment...
1996    Overall very difficult, but covers some intere...
1997    Assignments are both challenging and very fun....
1998          Study for this. Very useful for interviews.
1999    Teaches you how to solve 90% of tough intervie...
Name: reviews, Length: 2000, dtype: object

In [15]:
df['reviews']= df['reviews'].apply(lambda x: clean_text(x))

In [16]:
df['reviews']

0                                 go office hour practice
1       one least favourite course although thing nice...
2       start low pace midterm get really fast cannot ...
3       take program experience ever start pretty slow...
4       love everything great instructor fair assignme...
                              ...                        
1995    useful course undergrad assignment hard heavil...
1996          overall difficult cover interesting concept
1997    assignment challenge fun exam somewhat brutal ...
1998                               study useful interview
1999    teach solve tough interview problem plain fant...
Name: reviews, Length: 2000, dtype: object

In [17]:
df['reviews']= df['reviews'].apply(lambda x: x.split())

In [18]:
df['reviews']

0                            [go, office, hour, practice]
1       [one, least, favourite, course, although, thin...
2       [start, low, pace, midterm, get, really, fast,...
3       [take, program, experience, ever, start, prett...
4       [love, everything, great, instructor, fair, as...
                              ...                        
1995    [useful, course, undergrad, assignment, hard, ...
1996    [overall, difficult, cover, interesting, concept]
1997    [assignment, challenge, fun, exam, somewhat, b...
1998                           [study, useful, interview]
1999    [teach, solve, tough, interview, problem, plai...
Name: reviews, Length: 2000, dtype: object

In [19]:
import gensim

In [21]:
model_w2v= gensim.models.Word2Vec(
    df['reviews'],
    vector_size= 200, # Embedding size, dimensions size
    window = 5, # Predict the woed on basis of 5 neighboring words
    sg= 1, # Skip Gram
    min_count = 2, # Don't learn words having frequency 2.
    workers= 8 # 
)
model_w2v.train(df['reviews'], total_examples= len(df['reviews']), epochs= 30)

(754848, 1021290)

In [22]:
model_w2v.wv['assignment'] # Weights of word

array([ 0.18086281,  0.1765539 , -0.24602327,  0.24146524,  0.28951547,
        0.14368783,  0.2956964 ,  0.30010724, -0.06362466, -0.0950039 ,
       -0.2671791 ,  0.3019463 , -0.2008132 ,  0.34258348, -0.12048569,
        0.12937596,  0.00195689,  0.28856406, -0.11838845, -0.2912196 ,
        0.1566854 , -0.0638738 , -0.15809946,  0.4107372 ,  0.00510668,
       -0.06625225,  0.06421471, -0.15289119, -0.02052864,  0.1802633 ,
        0.21283725,  0.05432767, -0.33093256, -0.22868629,  0.0633126 ,
       -0.04026464,  0.08879218,  0.02789182, -0.05457027, -0.01732863,
        0.14742206, -0.30382636, -0.25738472,  0.25935674,  0.37717155,
        0.05100643,  0.2467453 , -0.19775222,  0.22036117, -0.09001115,
        0.18715733, -0.04517519,  0.03282401, -0.16335385, -0.09986109,
       -0.1300207 , -0.00272858, -0.18705612, -0.28871274, -0.18202022,
       -0.02477205, -0.26852283,  0.31330281, -0.2763333 , -0.23582256,
       -0.04635693,  0.05365926,  0.33610606,  0.03000969,  0.15

# Twitter Fake vs True Classification

In [57]:
true= pd.read_csv(r"X:\BIA\Practise\news\True.csv", nrows= 500)
fake= pd.read_csv(r"X:\BIA\Practise\news\Fake.csv", nrows= 500)

In [58]:
true.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [59]:
fake.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [60]:
true['category']= 1
fake['category']= 0

In [61]:
df= pd.concat([true, fake])

In [62]:
df

,title,text,subject,date,category
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",1
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",1
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",1
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",1
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",1
...,...,...,...,...,...
495,RNC Staffer Assigned To Finding Positive Stor...,Imagine for a moment that your job was to scou...,News,"August 25, 2017",0
496,"Trump Wants It Both Ways, But You Can’t Pass ...",Everybody s gotten used to Donald Trump contra...,News,"August 25, 2017",0
497,"REPORT: Mueller Hot On Trump’s Trail, Has Bui...",Donald Trump is very afraid of Special Counsel...,News,"August 25, 2017",0
498,House Dem Wants GOP On Record: Stop Gov’t Spe...,"Early next month, the GOP-controlled House of ...",News,"August 24, 2017",0


In [63]:
df['text']= df['text'].apply(lambda x: clean_text(x))

In [64]:
df['text']

0      washington reuters head conservative republica...
1      washington reuters transgender people allow fi...
2      washington reuters special counsel investigati...
3      washington reuters trump campaign adviser geor...
4      seattle/washington reuters president donald tr...
                             ...                        
495    imagine moment job scour medium search rare ne...
496    everybody gotten use donald trump contradict a...
497    donald trump afraid special counsel robert mue...
498    early next month gop-controlled house represen...
499    know conservatives constantly scream fake news...
Name: text, Length: 1000, dtype: object

In [78]:
from keras.preprocessing import text, sequence
import numpy as np

In [67]:
tokenizer= text.Tokenizer(num_words= 5000)
tokenizer.fit_on_texts(df['text'])
tokenized_text= tokenizer.texts_to_sequences(df['text'])
tokenized_text= sequence.pad_sequences(tokenized_text, 300)
tokenized_text

array([[   6,   28,  322, ...,  386,   28,  734],
       [3672,  556, 1647, ...,   91, 1883, 1952],
       [   0,    0,    0, ...,  319,    5,    2],
       ...,
       [   0,    0,    0, ..., 2189,   84,   21],
       [   0,    0,    0, ...,  943,   84,   21],
       [   0,    0,    0, ...,   21,   55, 1740]])

In [85]:
EMBEDDING_FILE= r"X:\BIA\Practise\news\glove.twitter.27B.100d.txt"

In [86]:
def get_coefs(word, *arr):
    return word, np.asarray(arr, dtype= 'float32')

def load_embeddings(embedding_file, word_index, max_features):
    embeddings_index = {}
    with open(embedding_file, encoding='utf-8') as f:
        for line in f:
            word, coefs = get_coefs(*line.rstrip().rsplit(' '))
            embeddings_index[word] = coefs

    all_embs = np.stack(embeddings_index.values())
    emb_mean, emb_std = all_embs.mean(), all_embs.std()
    embed_size = all_embs.shape[1]

    nb_words = min(max_features, len(word_index))
    embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))
    for word, i in word_index.items():
        if i >= max_features:
            continue
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector

    return embedding_matrix, embed_size

In [87]:
embedding_matrix, embed_size= load_embeddings(EMBEDDING_FILE, tokenizer.word_index, max_features= 5000)

C:\Users\abdul\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py:3526: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [90]:
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [111]:
model= Sequential()

model.add(Embedding(5000, output_dim= embed_size, weights=[embedding_matrix], input_length=300, trainable= False))

model.add(LSTM(units= 128, return_sequences= True, recurrent_dropout= 0.25, dropout= 0.2))
model.add(LSTM(units= 64, recurrent_dropout= 0.25, dropout= 0.2))
model.add(Dense(units=32, activation= 'relu'))
model.add(Dense(units=1, activation= 'sigmoid'))

model.compile(optimizer= 'adam', loss='binary_crossentropy', metrics= ['accuracy'])

model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 300, 100)          500000    
                                                                 
 lstm_4 (LSTM)               (None, 300, 128)          117248    
                                                                 
 lstm_5 (LSTM)               (None, 64)                49408     
                                                                 
 dense_4 (Dense)             (None, 32)                2080      
                                                                 
 dense_5 (Dense)             (None, 1)                 33        
                                                                 
Total params: 668769 (2.55 MB)
Trainable params: 168769 (659.25 KB)
Non-trainable params: 500000 (1.91 MB)
_________________________________________________________________


In [112]:
X_train, X_test, y_train, y_test= train_test_split(tokenized_text, df['category'], stratify= df['category'], test_size= 0.2, random_state=42)

In [106]:
X_train.shape

(800, 300)

In [107]:
y_train.shape

(800,)

In [108]:
X_test.shape

(200, 300)

In [109]:
y_test.shape

(200,)

In [113]:
model.fit(X_train, y_train, batch_size=32, validation_data=(X_test, y_test), epochs=3, verbose=2)


Epoch 1/3


25/25 - 27s - loss: 0.3310 - accuracy: 0.8838 - val_loss: 0.0886 - val_accuracy: 0.9800 - 27s/epoch - 1s/step
Epoch 2/3
25/25 - 19s - loss: 0.0553 - accuracy: 0.9850 - val_loss: 0.0133 - val_accuracy: 1.0000 - 19s/epoch - 752ms/step
Epoch 3/3
25/25 - 19s - loss: 0.0372 - accuracy: 0.9900 - val_loss: 0.0076 - val_accuracy: 1.0000 - 19s/epoch - 758ms/step


In [116]:
y_pred= model.predict(X_test)
y_pred= (y_pred>0.5).astype('int32')
y_pred

7/7 [==============================] - 1s 79ms/step


array([[1],
       [0],
       [0],
       [0],
       [1],
       [1],
       [1],
       [1],
       [0],
       [0],
       [1],
       [1],
       [1],
       [0],
       [0],
       [0],
       [1],
       [0],
       [1],
       [0],
       [1],
       [0],
       [1],
       [0],
       [1],
       [1],
       [1],
       [1],
       [0],
       [1],
       [0],
       [1],
       [1],
       [1],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [1],
       [0],
       [0],
       [1],
       [0],
       [1],
       [1],
       [0],
       [1],
       [0],
       [1],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [1],
       [1],
       [1],
       [1],
       [0],
       [0],
       [0],
       [1],
       [1],
       [0],
       [0],
       [0],
       [0],
       [1],
       [1],
       [1],
       [1],
       [0],
       [0],
       [1],
       [0],
       [0],
       [1],
       [1],
    

In [115]:
y_test

407    1
293    0
217    0
54     0
114    1
      ..
336    1
12     1
343    0
94     1
252    1
Name: category, Length: 200, dtype: int64